In [1]:
from langchain import hub
from langchain.agents import create_structured_chat_agent, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage
from langchain.tools import BaseTool
from langchain_openai import ChatOpenAI

In [2]:
model = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

In [3]:
model.invoke([HumanMessage(content="'君不见黄河之水天上来奔流到海不复回'，这句话的字数是多少？")])

AIMessage(content='这句话一共有16个字。', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 42, 'total_tokens': 53}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-067a889a-6dfc-4730-91c5-e783f1a18311-0')

In [4]:
class TextLengthTool(BaseTool):
    name = "文本字数计算工具"
    description = "当你被要求计算文本的字数时，使用此工具"

    def _run(self, text):
        return len(text)

In [5]:
tools = [TextLengthTool()]

In [6]:
prompt = hub.pull("hwchase17/structured-chat-agent")
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: co

In [7]:
agent = create_structured_chat_agent(
    llm=model,
    tools=tools,
    prompt=prompt
)

In [8]:
memory = ConversationBufferMemory(
        memory_key='chat_history',
        return_messages=True
)

In [9]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, memory=memory, verbose=True, handle_parsing_errors=True
)

In [10]:
agent_executor.invoke({"input": "'君不见黄河之水天上来奔流到海不复回'，这句话的字数是多少？"})



> Entering new AgentExecutor chain...
{
  "action": "文本字数计算工具",
  "action_input": {
    "text": "君不见黄河之水天上来奔流到海不复回"
  }
}17{
  "action": "Final Answer",
  "action_input": "这句话的字数是17"
}

> Finished chain.


{'input': "'君不见黄河之水天上来奔流到海不复回'，这句话的字数是多少？",
 'chat_history': [HumanMessage(content="'君不见黄河之水天上来奔流到海不复回'，这句话的字数是多少？"),
  AIMessage(content='这句话的字数是17')],
 'output': '这句话的字数是17'}

In [11]:
agent_executor.invoke({"input": "请你充当我的物理老师，告诉我什么是量子力学"})



> Entering new AgentExecutor chain...
{
  "action": "Final Answer",
  "action_input": "量子力学是研究微观世界中原子和亚原子粒子行为的物理学分支。它描述了微观粒子的运动和相互作用，涉及概率性和波粒二象性等概念。"
}

> Finished chain.


{'input': '请你充当我的物理老师，告诉我什么是量子力学',
 'chat_history': [HumanMessage(content="'君不见黄河之水天上来奔流到海不复回'，这句话的字数是多少？"),
  AIMessage(content='这句话的字数是17'),
  HumanMessage(content='请你充当我的物理老师，告诉我什么是量子力学'),
  AIMessage(content='量子力学是研究微观世界中原子和亚原子粒子行为的物理学分支。它描述了微观粒子的运动和相互作用，涉及概率性和波粒二象性等概念。')],
 'output': '量子力学是研究微观世界中原子和亚原子粒子行为的物理学分支。它描述了微观粒子的运动和相互作用，涉及概率性和波粒二象性等概念。'}